<a href="https://colab.research.google.com/github/advaithhari/tensorflowPipeline/blob/main/testingPipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

     |████████████████████████████████| 1.5MB 4.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [1]:
pip install -q -U --use-deprecated=legacy-resolver tfx

     |████████████████████████████████| 2.3 MB 6.4 MB/s 
     |████████████████████████████████| 2.2 MB 29.6 MB/s 
     |████████████████████████████████| 1.3 MB 29.2 MB/s 
     |████████████████████████████████| 2.9 MB 49.6 MB/s 
     |████████████████████████████████| 1.7 MB 43.3 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 1.5 MB 51.0 MB/s 
     |████████████████████████████████| 1.2 MB 23.2 MB/s 
     |████████████████████████████████| 17.7 MB 91 kB/s 
     |████████████████████████████████| 103 kB 55.5 MB/s 
     |████████████████████████████████| 89 kB 4.6 MB/s 
     |████████████████████████████████| 394 kB 52.1 MB/s 
     |████████████████████████████████| 147 kB 43.1 MB/s 
     |████████████████████████████████| 9.0 MB 39.7 MB/s 
     |████████████████████████████████| 294 kB 48.2 MB/s 
     |████████████████████████████████| 784 kB 42.1 MB/s 
     |████████████████████████████████| 61 kB 5.2 MB/s 
     |████████████████

In [34]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input
import numpy as np
import pandas as pd


%reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [3]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.4.1
TFX version: 0.28.0


In [4]:
from google.colab import files
uploaded = files.upload()

Saving sign_mnist_test.csv to sign_mnist_test.csv


In [51]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://advaithhari.github.io/uploadtestfile/sign_mnist_test.csv'
data_filepath = os.path.join(_data_root, "sign_mnist_test.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-datavr90qxu4/data.csv', <http.client.HTTPMessage at 0x7f16ddad03d0>)

In [54]:
example_gen = CsvExampleGen(input=external_input("/tmp/tfx-datavr90qxu4/"))
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 4
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 4
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-03-18T21_00_56.321482-ggngn_yd/CsvExampleGen/examples/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:21777485,xor_checksum:1616101349,sum_checksum:1616101349"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [55]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2021-03-18T21_00_56.321482-ggngn_yd/CsvExampleGen/examples/4


In [56]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

Streaming output truncated to the last 5000 lines.
    value {
      int64_list {
        value: 175
      }
    }
  }
  feature {
    key: "pixel243"
    value {
      int64_list {
        value: 178
      }
    }
  }
  feature {
    key: "pixel244"
    value {
      int64_list {
        value: 180
      }
    }
  }
  feature {
    key: "pixel245"
    value {
      int64_list {
        value: 177
      }
    }
  }
  feature {
    key: "pixel246"
    value {
      int64_list {
        value: 144
      }
    }
  }
  feature {
    key: "pixel247"
    value {
      int64_list {
        value: 108
      }
    }
  }
  feature {
    key: "pixel248"
    value {
      int64_list {
        value: 155
      }
    }
  }
  feature {
    key: "pixel249"
    value {
      int64_list {
        value: 183
      }
    }
  }
  feature {
    key: "pixel25"
    value {
      int64_list {
        value: 168
      }
    }
  }
  feature {
    key: "pixel250"
    value {
      int64_list {
        value: 182


In [57]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 5
        type_id: 7
        uri: "/tmp/tfx-interactive-2021-03-18T21_00_56.321482-ggngn_yd/StatisticsGen/statistics/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [58]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [59]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 6
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 6
        type_id: 9
        uri: "/tmp/tfx-interactive-2021-03-18T21_00_56.321482-ggngn_yd/SchemaGen/schema/6"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [60]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'label',INT,required,single,-
'pixel1',INT,required,single,-
'pixel10',INT,required,single,-
'pixel100',INT,required,single,-
'pixel101',INT,required,single,-
...,...,...,...,...
'pixel95',INT,required,single,-
'pixel96',INT,required,single,-
'pixel97',INT,required,single,-


In [61]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 7
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 7
        type_id: 11
        uri: "/tmp/tfx-interactive-2021-03-18T21_00_56.321482-ggngn_yd/ExampleValidator/anomalies/7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [63]:
context.show(example_validator.outputs['anomalies'])

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)
